# Daily_Reports

In [ ]:
import smtplib
from email.message import EmailMessage
from schedule import every, repeat, run_pending
import time
import psycopg2
import csv
import os
from datetime import datetime

# Database connection parameters
db_params = {
    'dbname': 'College automated reporting system',
    'user': 'postgres',
    'password': 'Automated@12',
    'host': 'localhost',
    'port': 5432
}

# Base directory for images
image_base_dir = r'C:\Users\Prathamesh\Desktop\Infosys project\Image directory\2. Testing images\Multi face'
renamed_image_dir = r'C:\Users\Prathamesh\Desktop\Infosys project\Image directory\2. Testing images\renamed'

# Reconnect to the database
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# Function to fetch faculty emails
def get_faculty_emails():
    cursor.execute('SELECT email FROM "Clg automated"."Faculty"')
    rows = cursor.fetchall()
    return [row[0] for row in rows]

# Function to fetch subject names
def get_subject_names():
    cursor.execute('SELECT DISTINCT "name" FROM "Clg automated"."Subject"')
    rows = cursor.fetchall()
    return [row[0] for row in rows]

# Function to fetch subject names and their time ranges
def get_subject_time_ranges():
    cursor.execute('SELECT "name", "from_time", "to_time" FROM "Clg automated"."Subject"')
    rows = cursor.fetchall()
    return {row[0]: (row[1], row[2]) for row in rows}

# Function to preload and rename images
def preload_and_rename_images(subject_time_ranges):
    image_files = os.listdir(image_base_dir)
    preloaded_images = []

    for image_file in image_files:
        # Extract datetime from the filename
        datetime_str = os.path.splitext(image_file)[0]
        datetime_obj = datetime.strptime(datetime_str, "%Y%m%d%H%M%S")

        # Determine the subject name based on the time range
        subject_name = None
        for subject, (start_time, end_time) in subject_time_ranges.items():
            if start_time <= datetime_obj.time() <= end_time:
                subject_name = subject
                break

        if subject_name:
            new_filename = f"{subject_name}_{datetime_obj.strftime('%Y-%m-%d')}.jpg"
            new_filepath = os.path.join(renamed_image_dir, new_filename)

            # Copy and rename the image
            original_filepath = os.path.join(image_base_dir, image_file)
            with open(original_filepath, 'rb') as src, open(new_filepath, 'wb') as dst:
                dst.write(src.read())

            preloaded_images.append(new_filepath)

    return preloaded_images

# Preload subject time ranges and images at the start
subject_time_ranges = get_subject_time_ranges()
preloaded_images = preload_and_rename_images(subject_time_ranges)

# Query attendance data and generate daily report
def query_daily_attendance():
    query = '''
    SELECT s."Name" AS student_name, sb."name" AS subject_name
    FROM "Clg automated"."Attendance" a
    JOIN "Clg automated"."Student" s ON a.student_id = s.id
    JOIN "Clg automated"."Subject" sb ON a.subject_id = sb.id
    WHERE a.date = CURRENT_DATE
    ORDER BY sb."name", s."Name"
    '''
    cursor.execute(query)
    return cursor.fetchall()

def generate_csv(filename, header, data):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        # Transform the data into the desired format
        report_data = {subject: [] for subject in header}  # Initialize dictionary with subject names as keys

        for student_name, subject_name in data:
            report_data[subject_name].append(student_name)  # Append student name under the correct subject

        # Write the transformed data into the CSV file
        max_len = max(len(names) for names in report_data.values())
        for i in range(max_len):
            row = [report_data[subject][i] if i < len(report_data[subject]) else '' for subject in header]
            writer.writerow(row)

    return filename

def generate_daily_report():
    data = query_daily_attendance()
    subjects = get_subject_names()
    header = subjects  # Prepare header dynamically without "Student Name"
    return generate_csv("daily_attendance_report.csv", header, data)

def send_email(subject, body, to, attachments):
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = "support@aptpath.in"
    msg['To'] = to
    msg.set_content(body)
    
    for attachment in attachments:
        with open(attachment, 'rb') as file:
            file_data = file.read()
            file_name = os.path.basename(attachment)
        msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)
    
    with smtplib.SMTP('smtp.office365.com', 587) as smtp:
        smtp.starttls()
        smtp.login("support@aptpath.in", "btpdcnfkgjyzdndh")
        smtp.send_message(msg)

def send_daily_report():
    daily_report = generate_daily_report()
    faculty_emails = get_faculty_emails()
    attachments = [daily_report]
    
    # Include preloaded images as attachments
    attachments.extend(preloaded_images)
    
    for email in faculty_emails:
        send_email(
            subject="Daily Attendance Report",
            body="Please find the daily attendance report and images of the students attached.",
            to=email,
            attachments=attachments
        )

# Schedule the daily report generation and sending
@repeat(every().day.at("18:57"))
def job_daily():
    send_daily_report()

def start_scheduler():
    while True:
        run_pending()
        time.sleep(1)

if __name__ == "__main__":
    start_scheduler()

# Close the database connection
cursor.close()
conn.close()